compare calibration result and simulation result in the instant loop detector

In [109]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-02'
detector = 'e2w_in'
number = 100
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [110]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [111]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_in,1.577923e+09,enter,0_e2w_in,13.77,5.00,DEFAULT_VEHTYPE,None
3,e2w_in,1.577923e+09,enter,1_e2w_in,11.52,5.00,DEFAULT_VEHTYPE,3.61
6,e2w_in,1.577923e+09,enter,2_e2w_in,12.60,5.00,DEFAULT_VEHTYPE,1.94
8,e2w_in,1.577923e+09,enter,3_e2w_in,10.35,5.00,DEFAULT_VEHTYPE,2.99
10,e2w_in,1.577923e+09,enter,4_e2w_in,10.02,5.00,DEFAULT_VEHTYPE,1.42
...,...,...,...,...,...,...,...,...
240,e2w_in,1.577924e+09,enter,95_e2w_in,12.78,5.00,DEFAULT_VEHTYPE,16.81
242,e2w_in,1.577924e+09,enter,96_e2w_in,11.56,5.00,DEFAULT_VEHTYPE,2.04
245,e2w_in,1.577924e+09,enter,97_e2w_in,9.70,5.00,DEFAULT_VEHTYPE,26.16
247,e2w_in,1.577924e+09,enter,98_e2w_in,10.42,5.00,DEFAULT_VEHTYPE,32.41


In [112]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_in,1.577923e+09,12.910773,1.70,1577923226,1577923209,14.1610,13.333333,-0.08,-0.422560
1,1_e2w_in,1.577923e+09,11.554964,1.45,1577923230,1577923210,12.0785,11.944444,0.15,-0.389481
2,2_e2w_in,1.577923e+09,10.546810,1.40,1577923233,1577923211,11.6620,8.888889,-0.46,1.657921
3,3_e2w_in,1.577923e+09,9.748955,1.30,1577923235,1577923213,10.8290,8.888889,0.96,0.860066
4,4_e2w_in,1.577923e+09,9.906650,1.30,1577923237,1577923215,10.8290,8.611111,0.81,1.295539


In [113]:
print(len(data), len(df_instant_out))

100 100


compare time error and speed error in loop detector between simulation result and calibration result

In [114]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_in,1.577923e+09,13.77,1.577923e+09,12.910773,1577923226,13.333333
1,1_e2w_in,1.577923e+09,11.52,1.577923e+09,11.554964,1577923230,11.944444
2,2_e2w_in,1.577923e+09,12.60,1.577923e+09,10.546810,1577923233,8.888889
3,3_e2w_in,1.577923e+09,10.35,1.577923e+09,9.748955,1577923235,8.888889
4,4_e2w_in,1.577923e+09,10.02,1.577923e+09,9.906650,1577923237,8.611111


In [115]:
#compare_df = compare_df.head(19)

In [116]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.5681000018119812)

In [117]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.8515999817848205)

In [118]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(0.5064887685821816)

In [119]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(0.8428666666666668)

In [120]:
start = min(compare_df['time_sim_error'].min(),compare_df['time_calib_error'].min()) 
end = max(compare_df['time_sim_error'].max(), compare_df['time_calib_error'].max())


In [121]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [122]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-2.5799999237060547 4.789999961853027
-4.0 1.8599998950958252


In [123]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-1.761111111111111 3.7111111111111104
-1.2687288092974196 2.4175407641412843


In [124]:
start = min(compare_df['speed_sim_error'].min(),compare_df['speed_calib_error'].min()) 
end = max(compare_df['speed_sim_error'].max(), compare_df['speed_calib_error'].max())

In [125]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [126]:
#import os
#os.environ["HORNSGATAN_HOME"]
